In [1]:
import fastai2
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *

In [2]:
lbl2name = dict(
  n02086240= 'Shih-Tzu',
  n02087394= 'Rhodesian ridgeback',
  n02088364= 'Beagle',
  n02089973= 'English foxhound',
  n02093754= 'Australian terrier',
  n02096294= 'Border terrier',
  n02099601= 'Golden retriever',
  n02105641= 'Old English sheepdog',
  n02111889= 'Samoyed',
  n02115641= 'Dingo'
)

In [3]:
path = untar_data(URLs.IMAGEWOOF); path.ls()

(#3) [/home/lgvaz/.fastai/data/imagewoof/log.csv,/home/lgvaz/.fastai/data/imagewoof/val,/home/lgvaz/.fastai/data/imagewoof/train]

In [4]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
          get_items=get_image_files,
          splitter=GrandparentSplitter(valid_name='val'),
          get_y=[parent_label, lbl2name.get])

In [5]:
dbunch = dblock.databunch(path,
                          bs=64,
#                           item_tfms=[FlipItem(), RandomResizedCrop(128)],
                          item_tfms=[FlipItem(), Resize(128)],
                          batch_tfms=[Normalize(*imagenet_stats)])

In [6]:
class ResBlock(nn.Module):
    "Resnet block from `ni` to `nh` with `stride`"
    @delegates(ConvLayer.__init__)
    def __init__(self, expansion, ni, nh, stride=1, sa=False, sym=False,
                 norm_type=NormType.Batch, act_cls=defaults.activation, ndim=2, **kwargs):
        super().__init__()
        norm2 = NormType.BatchZero if norm_type==NormType.Batch else norm_type
        nf,ni = nh*expansion,ni*expansion
        layers  = [ConvLayer(ni, nh, 3, stride=stride, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nf, 3, norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        ] if expansion == 1 else [
                   ConvLayer(ni, nh, 1, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nh, 3, stride=stride, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nf, 1, norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        ]
        self.convs = nn.Sequential(*layers)
        self.sa = SimpleSelfAttention(nf,ks=1,sym=sym) if sa else noop
        self.idconv = noop if ni==nf else ConvLayer(ni, nf, 1, act_cls=None, ndim=ndim, **kwargs)
        self.pool = noop if stride==1 else AvgPool(2, ndim=ndim, ceil_mode=True)
        self.act = defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls()

    def forward(self, x): return self.act(self.sa(self.convs(x)) + self.idconv(self.pool(x)))

In [7]:
class XResNet2(nn.Sequential):
    def __init__(self, expansion, layers, c_in=3, c_out=1000, sa=False, sym=False, act_cls=defaults.activation):
        stem = []
        sizes = [c_in,16,32,64] if c_in<3 else [c_in,32,64,64]
        for i in range(3):
            stem.append(ConvLayer(sizes[i], sizes[i+1], stride=2 if i==0 else 1, act_cls=act_cls))

        block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
        blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2,
                                  sa = sa if i==len(layers)-4 else False, sym=sym, act_cls=act_cls)
                  for i,l in enumerate(layers)]
        super().__init__(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *blocks,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(block_szs[-1]*expansion, c_out),
        )
        init_cnn(self)

    def _make_layer(self, expansion, ni, nf, blocks, stride, sa, sym, act_cls):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1,
                      sa if i==blocks-1 else False, sym=sym, act_cls=act_cls)
              for i in range(blocks)])
      
fastai2.vision.models.xresnet.XResNet = XResNet2

In [8]:
def opt_func(*args, **kwargs):
  opt = RAdam(*args, mom=0.95, wd=1e-2, eps=1e-6, **kwargs)
  return Lookahead(opt)

In [9]:
m = xresnet50(c_out=dbunch.c, sa=True, sym=False, act_cls=MishJit)

In [10]:
learn = Learner(dbunch, m, LabelSmoothingCrossEntropy(), metrics=[accuracy], opt_func=opt_func)

## Results

### Best

In [11]:
# act stem+filter size+sa, 0.7, 0.7, 0.696, 0.68, 0.7, 0.72
learn.fit_flat_cos(5, 4e-3, pct_start=.72)

epoch,train_loss,valid_loss,accuracy,time
0,1.954807,1.991632,0.336000,01:07
1,1.668017,2.504525,0.296000,01:06
2,1.473064,1.423339,0.598000,01:02
3,1.336725,1.374938,0.610000,01:01
4,1.127977,1.162369,0.722000,01:04


### Others

In [11]:
# act stem + filter size, 0.67
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.956344,2.105358,0.306000,01:02
1,1.715564,2.018951,0.314000,00:55
2,1.560458,1.547535,0.516000,00:54
3,1.426260,1.507091,0.556000,00:54
4,1.230344,1.252019,0.674000,00:54


In [11]:
# act stem, 0.64, 0.66
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.931011,2.101542,0.312000,01:01
1,1.688755,2.109970,0.364000,00:56
2,1.553915,1.531896,0.540000,00:54
3,1.399058,1.523111,0.564000,00:54
4,1.224818,1.248687,0.666000,00:56


In [9]:
# default, 0.65, 0.64
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.954447,2.488014,0.268000,01:02
1,1.783182,1.990843,0.338000,00:55
2,1.613038,1.576200,0.502000,00:52
3,1.464347,1.480333,0.560000,00:58
4,1.259610,1.297685,0.648000,00:59
